In [4]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
import numpy as np

In [5]:
#retrieving the url for FOMC Statement
html_page= requests.get('https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm') 
soup= BeautifulSoup(html_page.content, 'html.parser')

In [6]:
#preview the soup... MMM soup
print(soup.prettify()[:1000])

<!DOCTYPE doctype html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0 maximum-scale=1.6, user-scalable=1" name="viewport"/>
  <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" name="description">
   <meta content="Federal Reserve issues FOMC statement" property="og:title">
    <meta content="Board of Governors of the Federal Reserve System" property="og:site_name">
     <meta content="article" property="og:type"/>
     <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" property="og:description"/>
     <meta content="" property="og:image"/>
     <meta content="summary" name="twitter:card"/>


In [7]:
#search for the text in the article, isolate and save text
article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()

In [8]:
article

"\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In support of these goals, the Committee decided to ma

In [9]:
r = requests.get('https://www.federalreserve.gov/json/ne-press.json')

In [10]:
c = r.content

In [11]:
json_file = BeautifulSoup(c)

In [12]:
json.loads(json_file.text[0:3579] + ']')

[{'d': '3/25/2019 2:00:00 PM',
  't': 'Federal and state financial regulatory agencies issue interagency statement on supervisory practices regarding financial institutions affected by flooding in the Midwest',
  'pt': 'Banking and Consumer Regulatory Policy',
  'l': '/newsevents/pressreleases/bcreg20190325a.htm'},
 {'d': '3/22/2019 5:00:00 PM',
  't': 'Federal Reserve Board announces approval of proposals by First Interstate Bank',
  'pt': 'Orders on Banking Applications',
  'l': '/newsevents/pressreleases/orders20190322a.htm'},
 {'d': '3/22/2019 12:00:00 PM',
  't': 'Federal Reserve System publishes annual financial statements',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190322a.htm'},
 {'d': '3/21/2019 12:00:00 PM',
  't': 'Federal Reserve Board publishes report containing summary information on debit card transactions in 2017',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190321a.htm'},
 {'d': '3/21/2019 11:00:00 AM',
  't': '

In [13]:
our_list = json_file.text.split("Federal Reserve issues FOMC statement")

In [14]:
links = []
for element in our_list[1:]:
    links.append('https://www.federalreserve.gov/newsevents' + element.split('newsevents')[1][0:36])
links

['https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20190130a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181219a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181108a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180926a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180801a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180613a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180502a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180321a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171213a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171101a.htm',
 'https://www.fe

In [15]:
len('/pressreleases/bcreg20190325a.htm')

33

In [16]:
documents=[]
for link in links:
    html_page= requests.get(link) 
    soup= BeautifulSoup(html_page.content, 'html.parser')
    article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()
    documents.append(article)

In [17]:
listoftags=[]
for doc in documents:
    listoftags.append([doc,'dovish'])

In [18]:
listoftags[0][1]

'dovish'

In [19]:
df= pd.DataFrame(listoftags)

In [20]:
df.to_excel("FOMC.xlsx")

In [21]:
df=pd.read_excel("FOMC copy.xlsx")

In [22]:
df.drop(columns = 'Unnamed: 2', inplace= True)

In [23]:
df.columns = ["documents", "label"]

In [24]:
df.describe()

,documents,label
count,63,63
unique,63,2
top,\n\n Information received since the Fede...,dovish
freq,1,53


In [25]:
df.groupby('label').describe()

documents                                                            \
            count unique                                                top   
label                                                                         
dovish         53     53  \n\n       Information received since the Fede...   
hawkish        10     10  \nInformation received since the Federal Open ...   

              
        freq  
label         
dovish     1  
hawkish    1

In [26]:
def clean_data(data):
    cleaned_data = []
    for line in data:
        if not '[' in line and not ']' in line:
            for symbol in ",.?!''\n":
                line = line.replace(symbol, '').lower()
            cleaned_data.append(line)
    return "".join(cleaned_data)
clean_data(data_without_brackets)

NameError: name 'data_without_brackets' is not defined

In [28]:
df.documents[1] == df.documents[2] 

False

In [29]:
from nltk.corpus import stopwords

In [30]:
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [31]:
import string
def text_process(mess):
    # Check characters to see if they are in punctuation
    nopunc = [doc for doc in mess if doc not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [32]:
df.head()

,documents,label
0,\nInformation received since the Federal Open ...,dovish
1,\nInformation received since the Federal Open ...,dovish
2,\nInformation received since the Federal Open ...,hawkish
3,\nInformation received since the Federal Open ...,dovish
4,\nInformation received since the Federal Open ...,hawkish


In [33]:
df['documents'].head(10).apply(text_process)

0    [Information, received, since, Federal, Open, ...
1    [Information, received, since, Federal, Open, ...
2    [Information, received, since, Federal, Open, ...
3    [Information, received, since, Federal, Open, ...
4    [Information, received, since, Federal, Open, ...
5    [Information, received, since, Federal, Open, ...
6    [Information, received, since, Federal, Open, ...
7    [Information, received, since, Federal, Open, ...
8    [Information, received, since, Federal, Open, ...
9    [Information, received, since, Federal, Open, ...
Name: documents, dtype: object

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
df.head()

,documents,label
0,\nInformation received since the Federal Open ...,dovish
1,\nInformation received since the Federal Open ...,dovish
2,\nInformation received since the Federal Open ...,hawkish
3,\nInformation received since the Federal Open ...,dovish
4,\nInformation received since the Federal Open ...,hawkish


In [36]:
from imblearn.over_sampling import SMOTE, ADASYN
print(df['label'].value_counts()) #Previous original class distribution
X_resampled, y_resampled = SMOTE().fit_sample(df['documents'], df['label']) 

dovish     53
hawkish    10
Name: label, dtype: int64


ValueError: could not convert string to float: "\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In supp

In [37]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df['documents'])

print(len(bow_transformer.vocabulary_))
bow_transformer.vocabulary_

1067


{'Information': 126,
 'received': 844,
 'since': 930,
 'Federal': 103,
 'Open': 171,
 'Market': 153,
 'Committee': 74,
 'met': 684,
 'January': 132,
 'indicates': 597,
 'labor': 628,
 'market': 667,
 'remains': 868,
 'strong': 976,
 'growth': 551,
 'economic': 445,
 'activity': 241,
 'slowed': 937,
 'solid': 946,
 'rate': 832,
 'fourth': 531,
 'quarter': 825,
 'Payroll': 176,
 'employment': 460,
 'little': 647,
 'changed': 333,
 'February': 102,
 'job': 620,
 'gains': 537,
 'average': 297,
 'recent': 845,
 'months': 701,
 'unemployment': 1036,
 'remained': 866,
 'low': 653,
 'Recent': 192,
 'indicators': 599,
 'point': 779,
 'slower': 938,
 'household': 567,
 'spending': 952,
 'business': 322,
 'fixed': 519,
 'investment': 617,
 'first': 517,
 '12month': 6,
 'basis': 305,
 'overall': 745,
 'inflation': 600,
 'declined': 401,
 'largely': 632,
 'result': 881,
 'lower': 654,
 'energy': 463,
 'prices': 799,
 'items': 619,
 'food': 524,
 'near': 711,
 '2': 15,
 'percent': 766,
 'balance': 3

In [38]:
documents_bow = bow_transformer.transform(df['documents'])

In [39]:
print('Shape of Sparse Matrix: ', documents_bow.shape)
print('Amount of Non-Zero occurences: ', documents_bow.nnz)

Shape of Sparse Matrix:  (63, 1067)
Amount of Non-Zero occurences:  14290


In [40]:
sparsity = (100.0 * documents_bow.nnz / (documents_bow.shape[0] * documents_bow.shape[1]))
print('sparsity:', round(sparsity))

sparsity: 21


In [41]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(documents_bow)

In [42]:
documents_tfidf = tfidf_transformer.transform(documents_bow)
print(documents_tfidf)

  (0, 1060)	0.041389044804090946
  (0, 1043)	0.04428037501621468
  (0, 1036)	0.03202146992948094
  (0, 1018)	0.052049666442273934
  (0, 1005)	0.09023688897011548
  (0, 1002)	0.03410040147181566
  (0, 1000)	0.1343530958883752
  (0, 999)	0.05628725634353538
  (0, 997)	0.051078674109008855
  (0, 993)	0.09167858704452532
  (0, 976)	0.12522348704066175
  (0, 961)	0.029612612989913974
  (0, 953)	0.029612612989913974
  (0, 952)	0.03055277490603717
  (0, 946)	0.11731506013707327
  (0, 938)	0.09514452405062326
  (0, 937)	0.0671765479441876
  (0, 933)	0.04208584431735919
  (0, 930)	0.030078962990038493
  (0, 908)	0.030078962990038493
  (0, 881)	0.1322421090047724
  (0, 868)	0.06505536153766353
  (0, 866)	0.07878386451269744
  (0, 863)	0.054092725644679894
  (0, 845)	0.06505536153766353
  :	:
  (62, 119)	0.030148427835373703
  (62, 115)	0.027850521075420187
  (62, 113)	0.09975961678277745
  (62, 112)	0.05194919708927474
  (62, 105)	0.04488093618889678
  (62, 103)	0.024569695428493136
  (62, 100)	

In [43]:
!pip install imblearn

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(documents_tfidf, df['label'])

In [45]:
all_predictions = model.predict(documents_tfidf)
print(all_predictions)

['dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish']


In [46]:
from sklearn.metrics import classification_report
print(classification_report(df['label'], all_predictions))

              precision    recall  f1-score   support

      dovish       0.84      1.00      0.91        53
     hawkish       0.00      0.00      0.00        10

   micro avg       0.84      0.84      0.84        63
   macro avg       0.42      0.50      0.46        63
weighted avg       0.71      0.84      0.77        63



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = train_test_split(df['documents'], df['label'], test_size=0.2)

In [48]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [49]:
# k_fold = KFold(n_splits = 10)
# nb_f1_scores = []
# nb_conf_mat = np.array([[0, 0], [0, 0]])

# for train_indices, test_indices in k_fold.split(data):

#     train_text = data.iloc[train_indices]['sentence'].values
#     train_y = data.iloc[train_indices]['isRelevant'].values

#     test_text = data.iloc[test_indices]['sentence'].values
#     test_y = data.iloc[test_indices]['isRelevant'].values

#     vectorized_text = nb_pipeline.fit_transform(train_text)

#     sm = SMOTE(ratio = 1.0)
#     train_text_res, train_y_res = sm.fit_sample(vectorized_text, train_y)

#     clf = MultinomialNB()
#     clf.fit(train_text_res, train_y_res)
#     predictions = clf.predict(nb_pipeline.transform(test_text))

In [50]:
pipeline.fit(msg_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x10a34c488>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [51]:
predictions = pipeline.predict(msg_test)

In [52]:
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

      dovish       1.00      1.00      1.00        13

   micro avg       1.00      1.00      1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [54]:
df.head()

,documents,label
0,\nInformation received since the Federal Open ...,dovish
1,\nInformation received since the Federal Open ...,dovish
2,\nInformation received since the Federal Open ...,hawkish
3,\nInformation received since the Federal Open ...,dovish
4,\nInformation received since the Federal Open ...,hawkish


In [55]:
target = df['label']
df.drop("label", axis=1, inplace=True)

In [56]:
df.dtypes

documents    object
dtype: object

In [57]:
data = pd.get_dummies(df)
data.head()

documents_\n\n       Following careful deliberations at its recent meetings, the Federal Open Market Committee (FOMC) has reached broad agreement on the following principles regarding its longer-run goals and monetary policy strategy. The Committee intends to reaffirm these principles and to make adjustments as appropriate at its annual organizational meeting each January.\n    \n\n       The FOMC is firmly committed to fulfilling its statutory mandate from the Congress of promoting maximum employment, stable prices, and moderate long-term interest rates. The Committee seeks to explain its monetary policy decisions to the public as clearly as possible. Such clarity facilitates well-informed decisionmaking by households and businesses, reduces economic and financial uncertainty, increases the effectiveness of monetary policy, and enhances transparency and accountability, which are essential in a democratic society.\n    \n\n       Inflation, employment, and long-term interest rates fluctuate over time in response to economic and financial disturbances. Moreover, monetary policy actions tend to influence economic activity and prices with a lag. Therefore, the Committee's policy decisions reflect its longer-run goals, its medium-term outlook, and its assessments of the balance of risks, including risks to the financial system that could impede the attainment of the Committee's goals.\n    \n\n       The inflation rate over the longer run is primarily determined by monetary policy, and hence the Committee has the ability to specify a longer-run goal for inflation. The Committee judges that inflation at the rate of 2 percent, as measured by the annual change in the price index for personal consumption expenditures, is most consistent over the longer run with the Federal Reserve's statutory mandate. Communicating this inflation goal clearly to the public helps keep longer-term inflation expectations firmly anchored, thereby fostering price stability and moderate long-term interest rates and enhancing the Committee's ability to promote maximum employment in the face of significant economic disturbances.\n    \n\n       The maximum level of employment is largely determined by nonmonetary factors that affect the structure and dynamics of the labor market. These factors may change over time and may not be directly measurable. Consequently, it would not be appropriate to specify a fixed goal for employment; rather, the Committee's policy decisions must be informed by assessments of the maximum level of employment, recognizing that such assessments are necessarily uncertain and subject to revision. The Committee considers a wide range of indicators in making these assessments. Information about Committee participants' estimates of the longer-run normal rates of output growth and unemployment is published four times per year in the FOMC's Summary of Economic Projections. For example, in the most recent projections, FOMC participants' estimates of the longer-run normal rate of unemployment had a central tendency of 5.2 percent to 6.0 percent, roughly unchanged from last January but substantially higher than the corresponding interval several years earlier.\n    \n\n       In setting monetary policy, the Committee seeks to mitigate deviations of inflation from its longer-run goal and deviations of employment from the Committee's assessments of its maximum level. These objectives are generally complementary.  However, under circumstances in which the Committee judges that the objectives are not complementary, it follows a balanced approach in promoting them, taking into account the magnitude of the deviations and the potentially different time horizons over which employment and inflation are projected to return to levels judged consistent with its mandate.\n    \n\n        \n    \n\n        \n    \n\n        \n    \n  \
0                                                  0                                                                        

In [58]:
data_train, data_test, target_train, target_test = train_test_split(data, target, 
                                                                   test_size = 0.25, random_state=123)

In [59]:
tree_clf = DecisionTreeClassifier(criterion = "gini", max_depth = 5) 
tree_clf.fit(data_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [60]:
tree_clf.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.21860465, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.19939394, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.19072464, 0.20866808, 0.        , 0.1826087 , 0.        ,
       0.        , 0.        , 0.        ])

In [61]:
import matplotlib.pyplot as plt

In [62]:
pred = tree_clf.predict(data_test)
print(confusion_matrix(target_test, pred))
print(classification_report(target_test, pred))

[[14  0]
 [ 2  0]]
              precision    recall  f1-score   support

      dovish       0.88      1.00      0.93        14
     hawkish       0.00      0.00      0.00         2

   micro avg       0.88      0.88      0.88        16
   macro avg       0.44      0.50      0.47        16
weighted avg       0.77      0.88      0.82        16



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [63]:
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), n_estimators=20)

In [64]:
bagged_tree.fit(data_train, target_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=20, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [65]:
bagged_tree.score(data_train, target_train)

0.9574468085106383

In [66]:
bagged_tree.score(data_test, target_test)

0.875

In [67]:
forest = RandomForestClassifier(n_estimators=100, max_depth= 5)
forest.fit(data_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [68]:
forest.score(data_train, target_train)

0.8297872340425532

In [69]:
forest.score(data_test, target_test)

0.875

In [70]:
forest_2 = RandomForestClassifier(n_estimators = 5, max_features= 10, max_depth= 2)
forest_2.fit(data_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [81]:
forest.predict(df['documents'])

ValueError: could not convert string to float: "\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In supp